In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle
import holidays

with open('/content/drive/MyDrive/Colab Notebooks/GBDS_Project/raw_data/station_encoder.pkl', 'rb') as handle:
    le = pickle.load(handle)

In [ ]:
import holidays
us_holidays = holidays.country_holidays('US')

In [ ]:
standard_stations = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/standard_stations.csv").drop(columns=['Unnamed: 0'])
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/df_final_dataframe.csv").drop(columns=['Unnamed: 0'])

In [ ]:
distance = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/distance_matrix.csv").drop(columns=['Unnamed: 0'])

In [ ]:
allowed_stations = standard_stations.query("clusters==1")['station_id']
distance = distance[distance['end_station_id'].isin(allowed_stations) & distance['start_station_id'].isin(allowed_stations)].reset_index(drop=True)

In [ ]:
df = df[(df['start_station_cluster'] == 1) & (df['end_station_cluster'] == 1)].reset_index(drop=True)

In [ ]:
df['start_station_id'] = le.transform(df['start_station_id'])
df['end_station_id'] = le.transform(df['end_station_id'])
distance['start_station_id'] = le.transform(distance['start_station_id'])

distance['end_station_id'] = le.transform(distance['end_station_id'])


In [ ]:
distance['edge'] = distance['distance'] < 500
edge_index = distance[distance['edge'] == True][["start_station_id", 'end_station_id']].values
edge_index = edge_index.transpose()

In [ ]:
edge_index.shape

(2, 15472)

In [ ]:
column_name = 'started_at'
conversion_dict_hourly = dict(year= df[f'{column_name}_year'],
                              month=df[f'{column_name}_month'],
                              day=  df[f'{column_name}_day'],
                              hour= df[f'{column_name}_hour']
                              )
df[f'{column_name}_hourly'] = pd.to_datetime(conversion_dict_hourly)

In [ ]:
def demand_features(df):
  # Find out how many outgoing bikers we have
  outgoing_trips = df.groupby("start_station_id")["ride_id"].count()
  incoming_trips = df.groupby("end_station_id")["ride_id"].count()

  outgoing_trips = outgoing_trips.reindex(outgoing_trips.index.union(incoming_trips.index), fill_value=0).sort_index()
  incoming_trips = incoming_trips.reindex(outgoing_trips.index, fill_value=0).sort_index()

  # Build node features
  demand_features = np.stack([outgoing_trips, incoming_trips]).transpose()

  return demand_features

In [ ]:
node_features = demand_features(df)

In [ ]:
node_features.shape

(370, 2)

In [ ]:
def extract_dynamic_edges(s):
    # Extract dynamic edges and their features
    trip_indices = s[["start_station_id", "end_station_id"]].values
    trip_durations = s["duration_sec"]

    # Build edge features
    distance_feature  = pd.DataFrame(trip_indices,
                                    columns=["start_station_id", "end_station_id"]).merge(
                                        distance, on=["start_station_id", "end_station_id"],
                                        how="left")["distance"].values

    weather_features  = s[["temp", "prcp", "wspd", "rhum"]].values
    edge_type_feature = np.ones_like(distance_feature) # 1 = dynamic
    trip_duration_feature = trip_durations

    edge_features = np.stack([distance_feature, edge_type_feature, trip_duration_feature]).transpose()
    edge_features = np.hstack([edge_features, weather_features])
    return edge_features, trip_indices.transpose()

In [ ]:
# Add edge features to indicate edge type
distance_feature = distance[distance["edge"] == True]["distance"].values
edge_type_feature = np.zeros_like(distance_feature)
static_edge_features = np.stack([
    distance_feature,
    edge_type_feature,
    edge_type_feature,
    edge_type_feature,
    edge_type_feature,
    edge_type_feature,
    edge_type_feature
    ]).transpose()
static_edge_features.shape # [num_edges x num_features]

(15472, 7)

In [ ]:
from datetime import datetime, timedelta

date_col = f'{column_name}_hourly'

start_date = df[date_col].min()
end_date = df[date_col].max()

interval = timedelta(minutes=60)

xs = []
edge_indices = []
ys = []
y_indices = []
edge_features = []
count = 0
month_count = 0
while start_date <= end_date:
    # 0 - 60 min
    current_snapshot = df[((start_date + interval) >= df[date_col])
                                  & (start_date <= df[date_col])].copy()

    # 60 - 120 min
    subsequent_snapshot = df[((start_date + 2*interval) >= df[date_col])
                                  & (start_date + interval <= df[date_col])].copy()

    current_snapshot = current_snapshot.drop(columns=[
        'started_at_hourly',
        'ride_id',
        'started_at',
        'ended_at'
    ]).groupby(["start_station_id", "end_station_id"]).mean().reset_index()

    demand = subsequent_snapshot.groupby(["start_station_id", "end_station_id"])['ride_id'].count().reset_index()
    subsequent_snapshot = subsequent_snapshot.drop(columns=[
        'started_at_hourly',
        'ride_id',
        'started_at',
        'ended_at'
    ]).groupby(["start_station_id", "end_station_id"]).mean().reset_index()

    edge_feats, additional_edge_index = extract_dynamic_edges(current_snapshot)
    exteneded_edge_index = np.concatenate([edge_index, additional_edge_index], axis=1)
    extended_edge_feats = np.concatenate([edge_feats], axis=0)

    y = demand['ride_id'].values
    y_index = subsequent_snapshot[["start_station_id", "end_station_id"]].values

    xs.append(node_features)
    edge_indices.append(exteneded_edge_index) # static + dynamic
    edge_features.append(extended_edge_feats) # static + dynamic
    ys.append(y) # dynamic
    y_indices.append(y_index.transpose()) # dynamic
    # Increment
    start_date += interval
    count+=1
    if count%730==1:
      month_count+=1
      if count == 30:
        print(sss)
      print(month_count)

1


KeyboardInterrupt: 

In [ ]:
count

46

In [ ]:
edge_features[i]

array([[8.70403304e+02, 1.00000000e+00, 4.31000000e+02, 1.22000000e+01,
        0.00000000e+00, 2.23000000e+01, 7.50000000e+01],
       [0.00000000e+00, 1.00000000e+00, 2.47000000e+02, 1.22000000e+01,
        0.00000000e+00, 2.23000000e+01, 7.50000000e+01],
       [2.81325776e+03, 1.00000000e+00, 3.90100000e+03, 1.22000000e+01,
        0.00000000e+00, 2.23000000e+01, 7.50000000e+01],
       [1.07568350e+03, 1.00000000e+00, 4.77000000e+02, 1.17000000e+01,
        0.00000000e+00, 1.66000000e+01, 7.70000000e+01],
       [9.57098984e+02, 1.00000000e+00, 4.44000000e+02, 1.22000000e+01,
        0.00000000e+00, 2.23000000e+01, 7.50000000e+01],
       [7.06007445e+02, 1.00000000e+00, 9.28500000e+02, 1.17000000e+01,
        0.00000000e+00, 1.66000000e+01, 7.70000000e+01],
       [2.90551383e+02, 1.00000000e+00, 5.45000000e+02, 1.22000000e+01,
        0.00000000e+00, 2.23000000e+01, 7.50000000e+01],
       [1.65695197e+03, 1.00000000e+00, 1.04300000e+03, 1.22000000e+01,
        0.00000000e+00, 2

In [ ]:
i = 2
print(f"""Example of graph snapshot {i}: \n
      Node feature shape: {xs[i].shape} \n
      Edge index shape: {edge_indices[i].shape} \n
      Edge feature shape: {edge_features[i].shape} \n
      Labels shape: {ys[i].shape} \n
      Labels mask shape: {y_indices[i].shape}
      """)

Example of graph snapshot 2: 

      Node feature shape: (370, 2) 

      Edge index shape: (2, 15516) 

      Edge feature shape: (44, 7) 
 
      Labels shape: (25,) 

      Labels mask shape: (2, 25)
      


In [ ]:
import numpy as np

In [22]:
m = np.load("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/graphs/graphs_156_clusters_2024_4_11__1.npy")

In [23]:
n = np.load("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/graphs/graphs_156_clusters_2024_4_11__0.npy")

In [24]:
o = np.load("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/graphs/graphs_156_clusters_2024_4_11__2.npy")

In [ ]:
o[0][0]

array([2.021e+03, 1.000e+00, 3.000e+00, 2.500e+01, 1.100e+01, 0.000e+00,
       1.110e+01, 5.000e+01, 7.400e+01, 0.000e+00])

In [26]:
n[0].sum()

40.0